<a href="https://colab.research.google.com/github/phillippsm/colab_project/blob/colab_dev/ACT_Notifiable_Invoices_Register.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Datasets
### Distribution Shapes
### Measures of Central Tendency
### Central Limit Theorem


In [0]:
# We need Pandas and NumPy for these exercises
import pandas as pd
import numpy as np


### ACT Government Open Data Portal
#### Notifiable Invoices Register

In [0]:
# Read the latest 1000 records from the Notifiable Invoices Register API
nir_act = pd.read_csv("https://www.data.act.gov.au/resource/kzmf-7uhp.csv", parse_dates={"invoice_received_dt": ['date_invoice_received'], "payment_dt": ['payment_date']})
# This also converts dates from String format to Date format

# And look at the columns
nir_act.head()

In [0]:
# Describe the dataset - look at the Counts, Means, Standard Deviation and Quartiles for this data
nir_act.describe()

In [0]:
# What is the most recent payment date?
nir_act.payment_dt.max()

In [0]:
# How long does it take to pay an invoice - the difference between payment date and the date invoice received
nir_act['payment_delay'] = nir_act.payment_dt - nir_act.invoice_received_dt
# Turn that into a count of days (not a timedelta)
nir_act['payment_delay_days'] = nir_act.payment_delay.astype('timedelta64[D]')
# and look at it
nir_act.payment_delay_days


## Distribution Shape
Plot to see what the distribution looks like

In [0]:
#  Get our days column into a separate dataframe
nir_days = nir_act.payment_delay_days.copy()
# plot this - I have decided to use 7 days per bin (a week per bin)
nir_days.plot.hist(by='payment_delay_days', bins=int(nir_days.count()/7))
# and show the measures of central tendency
nir_days.describe()

## Is the MEDIAN (50%) equal to the mean?
We can see from the shape that this is not a normal distribution

In [0]:
#  Get our days column into a separate dataframe
nir_amounts = nir_act.payment_amount.copy()
# plot this - I have decided to break the payments into 100 even payment amount intervals
nir_amounts.plot.hist(by='payment_amount', bins=int(nir_amounts.count()/10))
# and show the measures of central tendency, variation and distribution pattern
nir_amounts.describe()

Both distributions are skewed
# Discussion
## What causes the skew in this distribution?
*   What are we measuring?
*   What are the outliers?
*   What causes the outliers?
*   Is the mean near the peak (bin/mode)?
*   Is the median near the peak?  Or near the mean?
*   When discussing wealth distribution, or house prices, is the mean a useful measure?
*   When looking at time periods for payment what causes a skew?
*   Is a graph better than a list of numbers for understanding the pattern of a distribution?







## Central Limit Theorem

If we take smaller samples of this set of data values at random, and compute the mean of this sample; and repeat this a number of times - the resulting distribution of the MEANS of all the samples will form a normal distribution.

### Let's test this theory with the invoice values (more extremely diverse with outrageous outliers)

In [0]:
# CHECKING THE CENTRAL LIMIT THEOREM
# Create a variable for the number of means for our distribution 
# - smaller numbers will be less "normal"
# - larger numbers will be more normal
# - we only have 1000 data points - but we can randomly sample these as many times as we like!
clt_means_size = 32 # 32768 #16384 - some theories state that you only need 30 samples - but I don't think so
print(nir_amounts.count())
# How many elements will we sample each time?
# not more than the number of data points (must be less than 1000)
# not too small (again - 30 should be our minimum)
clt_sample_size = int(nir_amounts.count() / 10) # this will make for 100 samples
# create a set of zeroes the size of the number of means we will collect
zeroes = np.zeros(clt_means_size)
# turn the zeroes into a dataframe with a column called "means" - for the means we will put add
clt_explore = pd.DataFrame(zeroes, index=np.arange(len(zeroes)), columns=['means'])
# check that this has worked
clt_explore.head()
# for an index (i) between 0 and the number of means we wish to collect
for i in range(clt_means_size):
  # take a random sample of our original data, of our chosen sample size
  samp = nir_amounts.sample(clt_sample_size)
  # add the sample mean to our collection of means
  clt_explore.at[i,'means'] = samp.mean()

# DECISIONS FOR BIN SIZE - for plotting our sample means
if (clt_means_size < 200):
  bin_lim = 1
elif (clt_means_size < 2000):
  bin_lim = 2
else:
  bin_lim = clt_means_size / 1000
# Calculate the bin_count (how many samples per bin)
bin_count = int(clt_means_size/(np.log(clt_means_size)*bin_lim))
#  print(bin_count)  # print to check bin_count calculation if you like

# plot this as a histogram - should look normal-ish
clt_explore.plot.hist(bins=bin_count, figsize=(12,6))

# Tests for being NORMAL
# Get the 1st quartile cutoff
q1 = clt_explore.means.quantile(q=0.25)
# Get the 3rd quartile
q3 = clt_explore.means.quantile(q=0.75)
# q3 - q1 is the IQR (interquartile range)
iqr = q3 - q1
# std is the standard deviation (default is correct for a sample)
std = clt_explore.means.std()
# compare the standard deviation with the IQR 
# - for a normal distribution we want a value close to 1.34898
print ('IQR ratio to STD (should be around 1.34898): ',iqr/std)
# compare the IQR to the minimum value. The minimum value should be close to, 
# but hopefully just below, Q1 - (1.5* IQR)
print ('IQR compared to min (Q1 - 1.5*IQR): ', clt_explore.means.min(), ' ', q1-1.5*iqr)
# compare the IQR to the maximum value. The maximum value should be close to, 
# but hopefully just above, Q3 + (1.5* IQR)
print ('IQR compared to max (Q3 + 1.5*IQR): ', clt_explore.means.max(), ' ', q3+1.5*iqr)

# For a normal distribution, the mean, median and mode should align
# for continuous data we may have no mode
# so - we can cut the dataframe into bins (using the same bin count as for plotting)
# The first 5 "value_counts()" will show us the most popular bins
print('Mode (by most popular bins)')
print(pd.cut(clt_explore.means, bins=bin_count).value_counts().head())
# This should be around the same as the median and mean

# median == Q2 (the 50% quartile)
clt_explore.describe()
# We have also shown the "mean of our means", and the standard deviation of the mean of our samples


# Exercise for you

### do the same with the nir_days

Does the distribution change to a normal-ish distribution

Does the Central Limit Theorm apply in this context?

In [0]:
# Copy the "CHECKING THE CENTRAL LIMIT THEOREM" cell here
# and change nir_amounts to nir_days

# EXTRA 
## Time to Pay?
#### **looking for relationships between independent variables**
I decided to investigate the time taken to pay an invoice. Does it take longer for large amounts - I want to explore this relationship

Payment amounts can be huge.  Up to 4 million!  

But more often smaller.

This variability is problematic for establishing a relationship with "time to pay" as the range of those values is from 0 days to 400 days.  I deiced to investigate using a "log of payment amount".


In [0]:
# Create a column for the "naural logarithm" of the payment amount.
nir_act['payment_amount_ln'] = np.log(nir_act.payment_amount)
nir_act.payment_amount_ln

## Plot to see if there is a relationship
Plot payment_delays_days vs payment_amount_ln as a scatter plot

Does it show any relationship?


In [0]:
# Let's plot these to see if it looks like there is a relationship
# we add a filter to exclude the longest payment delays: 'nir_act.payment_delay_days<200.0'
# is it valid to filter to see the relationship?
nir_act[nir_act.payment_delay_days<200.0].plot.scatter(x='payment_delay_days', y='payment_amount_ln', figsize=(8,8))

In [0]:
# what if we filter for just the longest payment delays
nir_act[nir_act.payment_delay_days>300.0].head()

In the next investigation we will look at relationships between independent variables in more detail